In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Now we load in the data file.

In [2]:
original_data = pd.read_csv("../Data/lead_data.csv",
                        encoding="utf-8")
# Need encoding change for weird characters to come through
lead_data = original_data.copy()
lead_data.head(20)

,Unnamed: 0,Title,Job Function,Job Role,Job Level
0,0,MANAGER-CYBERSECURITY,IT,INFORMATION SECURITY,MANAGER
1,1,"MANAGER, INFORMATION SECURITY",IT,INFORMATION SECURITY,MANAGER
2,2,USER EXPERIENCE ANALYST,ENGINEERING,DEVELOPMENT,CONTRIBUTOR
3,3,NETWORK SPECIALIST,IT,NETWORKING,CONTRIBUTOR
4,4,DIRECTOR OF PRIVACY AND COMPLIANCE,IT,INFORMATION SECURITY,DIRECTOR
5,5,"ASSISTANT VP, NETWORK ARCHITECTURE CCIE 1676",IT,NETWORKING,CONTRIBUTOR
6,6,"DIRECTOR, INFORMATION SECURITY",IT,NETWORKING,DIRECTOR
7,7,CIO,IT,INFORMATION SECURITY,C-LEVEL
8,8,DIRECTOR CLOUD OPERATIONS,IT,IT GENERAL,DIRECTOR
9,9,"DIRECTOR, ENTERPRISE APPLICATIONS",IT,NETWORKING,DIRECTOR


Dimensions of data

In [3]:
lead_data.shape

(837232, 5)

Unique values count

In [4]:
lead_data.nunique()

Unnamed: 0      837232
Title           170880
Job Function         5
Job Role             7
Job Level            6
dtype: int64

Detailed summary

In [5]:
pd.set_option('display.max_rows',100)
for col in lead_data.drop('Title',axis = 1):
    lead_data[col].value_counts()

Unnamed: 0
0         1
558159    1
558149    1
558150    1
558151    1
         ..
279079    1
279080    1
279081    1
279082    1
837231    1
Name: count, Length: 837232, dtype: int64

Job Function
IT                       592609
NON-ICP                  161450
ENGINEERING               79119
PROCUREMENT                2486
RISK/LEGAL/COMPLIANCE      1568
Name: count, dtype: int64

Job Role
INFORMATION SECURITY          272404
NETWORKING                    216843
NON-ICP                       186470
DEVELOPMENT                    81663
IT GENERAL                     45253
GOVERNANCE RISK COMPLIANCE     20224
SYSTEMS                        14375
Name: count, dtype: int64

Job Level
CONTRIBUTOR    309800
MANAGER        174717
DIRECTOR       142376
EXECUTIVE      110363
C-LEVEL         96681
UNKNOWN          3295
Name: count, dtype: int64

In [6]:
pd.set_option('display.max_rows',100)
for col in lead_data.drop('Title',axis = 1):
    temp = lead_data[col].value_counts(normalize=True)
    temp
    temp.iloc[0:5].sum()

Unnamed: 0
0         0.000001
558159    0.000001
558149    0.000001
558150    0.000001
558151    0.000001
            ...   
279079    0.000001
279080    0.000001
279081    0.000001
279082    0.000001
837231    0.000001
Name: proportion, Length: 837232, dtype: float64

5.972060313031513e-06

Job Function
IT                       0.707819
NON-ICP                  0.192838
ENGINEERING              0.094501
PROCUREMENT              0.002969
RISK/LEGAL/COMPLIANCE    0.001873
Name: proportion, dtype: float64

1.0

Job Role
INFORMATION SECURITY          0.325363
NETWORKING                    0.259000
NON-ICP                       0.222722
DEVELOPMENT                   0.097539
IT GENERAL                    0.054051
GOVERNANCE RISK COMPLIANCE    0.024156
SYSTEMS                       0.017170
Name: proportion, dtype: float64

0.9586745370458846

Job Level
CONTRIBUTOR    0.370029
MANAGER        0.208684
DIRECTOR       0.170056
EXECUTIVE      0.131819
C-LEVEL        0.115477
UNKNOWN        0.003936
Name: proportion, dtype: float64

0.9960644122537122

Number of rows where any one of the outputs are NA

In [8]:
(lead_data['Job Role'].isna()|lead_data['Job Function'].isna()|lead_data['Job Level'].isna()|lead_data['Title'].isna()).sum()
(lead_data['Job Role'].isna()|lead_data['Job Function'].isna()|lead_data['Job Level'].isna()|lead_data['Title'].isna()).sum()/lead_data.shape[0]

0

0.0

In [9]:
words_series = lead_data['Title'].loc[~lead_data['Title'].isna()].str.lower().str.split().tolist()
words_list = [word for title in words_series for word in title]

Unique number of words

In [10]:
words_set = set(words_list)
len(words_set)

29279

Remove stopwords

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\csarc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
len(words_set - stop_words)

29190

Not that many overlapping, so there's not really that much of an impact of removing stopwords. Thus, for the model, I won't even bother to do this.

In [13]:
# Overlapping compliance
((lead_data['Job Function'] == 'RISK/LEGAL/COMPLIANCE') & (lead_data['Job Role'] == "GOVERNANCE RISK COMPLIANCE")).sum()
((lead_data['Job Function'] == 'RISK/LEGAL/COMPLIANCE') & (lead_data['Job Role'] != "GOVERNANCE RISK COMPLIANCE")).sum()
((lead_data['Job Function'] != 'RISK/LEGAL/COMPLIANCE') & (lead_data['Job Role'] == "GOVERNANCE RISK COMPLIANCE")).sum()
((lead_data['Job Function'] == 'RISK/LEGAL/COMPLIANCE') | (lead_data['Job Role'] == "GOVERNANCE RISK COMPLIANCE")).sum()

2

1566

20222

21790

In [14]:
# Function = IT but Role = NON-ICP
((lead_data['Job Function'] == 'IT') & (lead_data['Job Role'] == "NON-ICP")).sum()
((lead_data['Job Function'] == 'IT') & (lead_data['Job Role'] != "NON-ICP")).sum()
((lead_data['Job Function'] == 'IT')).sum()

27097

565512

592609